<a href="https://colab.research.google.com/github/Venkatpandey/DataScience_ML/blob/main/featureSelection/06.2-Method-used-in-a-KDD-competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Method used in a KDD 2009 competition

We will cover the feature selection approach undertaken by data scientists at the University of Melbourne in the [KDD 2009](http://www.kdd.org/kdd-cup/view/kdd-cup-2009) data science competition. The task consisted in predicting churn based on a dataset with a huge number of features.

The authors describe this procedure as an aggressive non-parametric feature selection procedure that is based in contemplating the relationship between the feature and the target. Therefore, this method should be classified as a filter method.

**The procedure consists in the following steps**:

For each categorical variable:

    1) Separate into train and test

    2) Determine the mean value of the target within each label of the categorical variable using the train set

    3) Use that mean target value per label as the prediction (using the test set) and calculate the roc-auc.

For each numerical variable:

    1) Separate into train and test
    
    2) Divide the variable into 100 quantiles

    3) Calculate the mean target within each quantile using the training set 

    4) Use that mean target value / bin as the prediction (using the test set) and calculate the roc-auc


The authors quote the following advantages of the method:

- Speed: computing mean and quantiles is direct and efficient
- Stability respect to scale: extreme values for continuous variables do not skew the predictions
- Comparable between categorical and numerical variables
- Accommodation of non-linearities

See my notes at the end of the notebook for a discussion on the method.

**Important**
The authors here use the roc-auc, but in principle, we could use any metric, including those valid for regression.

**Reference**:
[Predicting customer behaviour: The University of Melbourne's KDD Cup Report. Miller et al. JMLR Workshop and Conference Proceedings 7:45-55](http://www.mtome.com/Publications/CiML/CiML-v3-book.pdf)

## How to do it?

In [80]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score

In [81]:
# load the titanic dataset
data = pd.read_csv('https://raw.githubusercontent.com/Venkatpandey/DataScience_ML/main/dataset/titanic.csv')
data.shape

(1306, 9)

In [82]:
data.head(20)

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked
0,1,1,female,29.000000,0,0,211.3375,B5,S
1,1,1,male,0.916700,1,2,151.5500,C22 C26,S
2,1,0,female,2.000000,1,2,151.5500,C22 C26,S
3,1,0,male,30.000000,1,2,151.5500,C22 C26,S
4,1,0,female,25.000000,1,2,151.5500,C22 C26,S
5,1,1,male,48.000000,0,0,26.5500,E12,S
6,1,1,female,63.000000,1,0,77.9583,D7,S
7,1,0,male,39.000000,0,0,0.0000,A36,S
8,1,1,female,53.000000,2,0,51.4792,C101,S
9,1,0,male,71.000000,0,0,49.5042,NaN,C


In [83]:
# Variable preprocessing:

# then I will narrow down the different cabins by selecting only the
# first letter, which represents the deck in which the cabin was located

# captures first letter of string (the letter of the cabin)
data['cabin'] = data['cabin'].str[0]
data['cabin'].unique()

array(['B', 'C', 'E', 'D', 'A', nan, 'T', 'F', 'G'], dtype=object)

### Feature selection on categorical variables

First, I will demonstrate the feature selection procedure over categorical variables. The Titanic dataset contains 4 categorical variables, which are Sex, Pclass, Cabin and Embarked.

**Important**

In all feature selection procedures, it is good practice to select the features by examining only the training set. And this is to avoid overfit.

In [84]:
# separate train and test sets

# I will only use the categorical variables and the target

X_train, X_test, y_train, y_test = train_test_split(
    data[['pclass', 'sex', 'embarked', 'cabin', 'survived']],
    data['survived'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((914, 5), (392, 5))

In [85]:
# fill missing values

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

In [86]:
X_train.head(20)

,pclass,sex,embarked,cabin,survived
840,3,male,S,0,0
866,3,male,S,0,0
427,2,male,S,0,0
478,2,male,C,0,0
1305,3,male,S,0,0
453,2,male,S,0,0
117,1,female,C,E,1
482,2,female,S,F,1
294,1,female,C,C,1
261,1,male,S,E,0


### Replace categories by target mean

In [87]:
# function that determines the target mean per category

def mean_encoding(df_train, df_test, categorical_vars):
    
    # temporary copy of the original dataframes
    df_train_temp = df_train.copy()
    df_test_temp = df_test.copy()
    
    # iterate over each variable
    for col in categorical_vars:
        
        # make a dictionary of categories, target-mean pairs
        target_mean_dict = df_train.groupby([col])['survived'].mean().to_dict()
        
        # replace the categories by the mean of the target
        df_train_temp[col] = df_train[col].map(target_mean_dict)
        df_test_temp[col] = df_test[col].map(target_mean_dict)
    
    # drop the target from the daatset
    df_train_temp.drop(['survived'], axis=1, inplace=True)
    df_test_temp.drop(['survived'], axis=1, inplace=True)
    
    # return  remapped datasets
    return df_train_temp, df_test_temp

In [88]:
categorical_vars = ['pclass', 'sex', 'embarked','cabin']

X_train_enc, X_test_enc = mean_encoding(X_train, X_test, categorical_vars)

X_train_enc.head(20)

,pclass,sex,embarked,cabin
840,0.243902,0.199664,0.338534,0.295875
866,0.243902,0.199664,0.338534,0.295875
427,0.416667,0.199664,0.338534,0.295875
478,0.416667,0.199664,0.545946,0.295875
1305,0.243902,0.199664,0.338534,0.295875
453,0.416667,0.199664,0.338534,0.295875
117,0.639130,0.716981,0.545946,0.741935
482,0.416667,0.716981,0.338534,0.611111
294,0.639130,0.716981,0.545946,0.621622
261,0.639130,0.199664,0.338534,0.741935


In [89]:
X_test_enc.head()

,pclass,sex,embarked,cabin
609,0.243902,0.199664,0.338534,0.295875
412,0.416667,0.199664,0.338534,0.295875
528,0.416667,0.199664,0.338534,0.295875
1147,0.243902,0.716981,0.329545,0.295875
942,0.243902,0.199664,0.338534,0.295875


The strings were replaced by the target mean.

### Determine the roc-auc using the variable values as input

In [90]:
# now, we calculate a roc-auc value, using the encoded variables
# as predictions

roc_values = []

for feature in categorical_vars:
    
    roc_values.append(roc_auc_score(y_test, X_test_enc[feature])) 

In [91]:
# I make a series for easy visualisation

m1 = pd.Series(roc_values)
m1.index = categorical_vars
m1.sort_values(ascending=False)

sex         0.784164
pclass      0.630389
cabin       0.610934
embarked    0.573342
dtype: float64

We can see, that all the features are important, because the roc_auc for all of them is higher than 0.5.

Sex seems to be the most important feature to predict survival, as its roc_auc is the highest.

As you see, this is a very powerful, yet straightforward approach to feature selection.

## Feature Selection on numerical variables

The procedure is exactly the same, but it requires one additional first step which is to divide the continuous variable into bins. 

The authors of the method divide the variable in 100 quantiles, that is 100 bins. In principle, you could divide the variable in less bins. Here I will divide the variable in 5 bins only.

I will work with the numerical variables Age and Fare.

In [92]:
# separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data[['age', 'fare', 'survived']],
    data['survived'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((914, 3), (392, 3))

In [93]:
# fill missing values

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

#### Bin variable Age

In [94]:
# Let's divide Age in 10 bins. We use the qcut (quantile cut)
# function from pandas indicating that 11 cutting points (q),
# thus 10 bins.

# retbins= True indicates that I want to capture the limits of
# each interval (so I can then use them to cut the test set)

X_train['age_binned'], intervals = pd.qcut(
    X_train['age'],
    q = 5,
    labels=False,
    retbins=True,
    precision=3,
    duplicates='drop',
)

X_train[['age_binned', 'age']].head(10)

,age_binned,age
840,2,29.813199
866,4,43.000000
427,4,44.000000
478,1,25.000000
1305,1,29.000000
453,4,63.000000
117,3,30.000000
482,3,34.000000
294,3,39.000000
261,4,50.000000


In [95]:
# count the number of distinct bins

X_train['age_binned'].nunique()

5

In [96]:
# display the bins

X_train['age_binned'].unique()

array([2, 4, 1, 3, 0])

In [97]:
# now I use the interval limits calculated in the previous cell to
# bin the testing set

X_test['age_binned'] = pd.cut(x = X_test['age'], bins=intervals, labels=False)

X_test[['age_binned', 'age']].head(10)

,age_binned,age
609,0.0,0.833300
412,3.0,34.000000
528,0.0,19.000000
1147,2.0,29.813199
942,2.0,29.813199
870,2.0,29.813199
5,4.0,48.000000
231,4.0,47.000000
731,0.0,9.000000
1289,2.0,29.813199


#### Bin Variable Fare

In [98]:
# train
X_train['fare_binned'], intervals = pd.qcut(
    X_train['fare'],
    q=5,
    labels=False,
    retbins=True,
    precision=3,
    duplicates='drop',
)

# test
X_test['fare_binned'] = pd.cut(x = X_test['fare'], bins=intervals, labels=False)

In [99]:
X_test['fare_binned'].nunique()

5

In [100]:
X_test.isnull().sum()

age            0
fare           0
survived       0
age_binned     2
fare_binned    5
dtype: int64

In [101]:
# test shows some missing data. The missing values in 
# the test set appear when the original values are outside
# the boundaries of the invervals determined in the train set
# that is, values that are smaller or bigger than the min and max
# values from the train set

# to speed out the demo, I will just replace them by 0 in this notebook

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

### Replace bins with target mean

In [102]:
# now we use our previous function to encode the variables
# with the target mean

binned_vars = ['age_binned', 'fare_binned']

X_train_enc, X_test_enc = mean_encoding(
    X_train[binned_vars+['survived']], X_test[binned_vars+['survived']], binned_vars)

X_train_enc.head()

,age_binned,fare_binned
840,0.254237,0.367232
866,0.421965,0.256831
427,0.421965,0.367232
478,0.379487,0.629834
1305,0.379487,0.207447


### Determine roc-auc using encodings

In [103]:
# now, we calculate a roc-auc value, using the encoded variables
# as predictions

roc_values = []

for feature in binned_vars:
    
    roc_values.append(roc_auc_score(y_test, X_test_enc[feature])) 

In [104]:
# I make a series for easy visualisation

m1 = pd.Series(roc_values)
m1.index = binned_vars
m1.sort_values(ascending=False)

fare_binned    0.666964
age_binned     0.497335
dtype: float64

Fare, is a much better predictor of Survival. Age produces a random output, the roc-auc is 0.5.


**Some thoughts**

The authors mention that by using this method, you are able to compare directly numerical with categorical variables. In a sense this is true, however we need to keep in mind, that categorical variables may or may not (and typically they will not) show the same percentage of observations per label. However, when we divide a numerical variable into quantile bins, we guarantee that each bin shows the same percentage of observations.

Alternatively, instead of binning into quantiles, we can bin into equal-distance bins.